In [1]:
import os
import json
import pandas as pd
import glob

print("Libraries Import Successfully")

Libraries Import Successfully


In [3]:
electricity_folder = r"C:\Users\M. Faizan\Desktop\SE\6th\Data_Science_Assign2\raw\electricity_raw_data"
weather_folder = r"C:\Users\M. Faizan\Desktop\SE\6th\Data_Science_Assign2\raw\weather_raw_data"

In [5]:

# Load electricity data (JSON files)
electricity_data = []
for file_path in glob.glob(os.path.join(electricity_folder, "*.json")):
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
        records = data.get("response", {}).get("data", [])
        electricity_data.extend(records)

# Convert to DataFrame and standardize column names
electricity_df = pd.DataFrame(electricity_data)
electricity_df.rename(columns={"period": "datetime", "value": "electricity_demand"}, inplace=True)
electricity_df["datetime"] = pd.to_datetime(electricity_df["datetime"], format="%Y-%m-%dT%H", errors='coerce')

# Load weather data (CSV files)
weather_data = []
for file_path in glob.glob(os.path.join(weather_folder, "*.csv")):
    df = pd.read_csv(file_path, encoding="utf-8", header=0)
    df.columns = df.columns.str.strip()  # Remove leading/trailing spaces
    weather_data.append(df)

# Merge all weather data into a single DataFrame
weather_df = pd.concat(weather_data, ignore_index=True)
print("Weather Data Columns:", weather_df.columns)  # Debugging step

# Ensure the correct datetime column is used and convert to datetime format
if "datetime" in weather_df.columns:
    weather_df["datetime"] = pd.to_datetime(weather_df["datetime"], errors='coerce')
elif "timestamp" in weather_df.columns:
    weather_df.rename(columns={"timestamp": "datetime"}, inplace=True)
    weather_df["datetime"] = pd.to_datetime(weather_df["datetime"], errors='coerce')
elif "date" in weather_df.columns:  # Your dataset has 'date' instead of 'datetime'
    weather_df.rename(columns={"date": "datetime"}, inplace=True)
    weather_df["datetime"] = pd.to_datetime(weather_df["datetime"], errors='coerce')
else:
    raise KeyError("No valid datetime column found in weather data")

# Drop rows with invalid datetime values
weather_df.dropna(subset=["datetime"], inplace=True)

#  Fix: Ensure both datetime columns have the same format (remove timezone differences)
electricity_df["datetime"] = electricity_df["datetime"].dt.tz_localize(None)
weather_df["datetime"] = weather_df["datetime"].dt.tz_localize(None)

# Merge electricity and weather data on the datetime column
final_df = pd.merge(electricity_df, weather_df, on="datetime", how="inner")

#  Handling Duplicates and Inconsistencies
# Remove duplicate rows to maintain data integrity
final_df.drop_duplicates(inplace=True)

#  Ensure electricity_demand is numeric
final_df["electricity_demand"] = pd.to_numeric(final_df["electricity_demand"], errors="coerce")
final_df.dropna(subset=["electricity_demand"], inplace=True)  # Drop rows with NaN values

#  Identify and remove outliers in numerical columns using IQR method
Q1 = final_df["electricity_demand"].quantile(0.25)
Q3 = final_df["electricity_demand"].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Remove outliers
final_df = final_df[(final_df["electricity_demand"] >= lower_bound) & (final_df["electricity_demand"] <= upper_bound)]

#  Feature Engineering
# Extract additional temporal features for analysis
final_df["hour"] = final_df["datetime"].dt.hour
final_df["day"] = final_df["datetime"].dt.day
final_df["month"] = final_df["datetime"].dt.month
final_df["year"] = final_df["datetime"].dt.year
final_df["day_of_week"] = final_df["datetime"].dt.dayofweek
final_df["is_weekend"] = final_df["day_of_week"].apply(lambda x: 1 if x >= 5 else 0)
final_df["season"] = final_df["month"].apply(lambda x: ("Winter" if x in [12, 1, 2] else
                                                           "Spring" if x in [3, 4, 5] else
                                                           "Summer" if x in [6, 7, 8] else
                                                           "Fall"))

# Normalize numerical features (Min-Max Scaling)
numeric_cols = ["electricity_demand"]
for col in numeric_cols:
    final_df[col] = (final_df[col] - final_df[col].min()) / (final_df[col].max() - final_df[col].min())

# Convert categorical columns to category dtype for efficiency
categorical_cols = ["subba", "subba-name", "parent", "parent-name", "value-units", "season"]
for col in categorical_cols:
    if col in final_df.columns:
        final_df[col] = final_df[col].astype("category")

#  Ensure the processed directory exists
output_dir = r"C:\Users\M. Faizan\Desktop\SE\6th\Saved"
os.makedirs(output_dir, exist_ok=True)  # Create directory if it doesn't exist

#  Missing Data Analysis
missing_data = final_df.isnull().sum()
missing_percentage = (missing_data / len(final_df)) * 100
missing_data_path = os.path.join(output_dir, "missing_data_analysis.csv")
missing_data_df = pd.DataFrame({"Missing Values": missing_data, "Percentage": missing_percentage})

# Save missing data analysis only if missing values exist
if missing_data_df["Missing Values"].sum() > 0:
    missing_data_df.to_csv(missing_data_path, index=True)
    print(f"Missing data analysis saved to: {missing_data_path}")
else:
    print("No missing data found. File not created.")

# Save updated data
updated_data_path = os.path.join(output_dir, "cleaned_and_featured_data.csv")
final_df.to_csv(updated_data_path, index=False)
print(f"Updated data saved to: {updated_data_path}")

# Summary of the final dataset
print("Data Summary:")
print(final_df.info())

Weather Data Columns: Index(['date', 'temperature_2m'], dtype='object')
No missing data found. File not created.
Updated data saved to: C:\Users\M. Faizan\Desktop\SE\6th\Saved\cleaned_and_featured_data.csv
Data Summary:
<class 'pandas.core.frame.DataFrame'>
Index: 261035 entries, 0 to 1392802
Data columns (total 15 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   datetime            261035 non-null  datetime64[ns]
 1   subba               261035 non-null  category      
 2   subba-name          261035 non-null  category      
 3   parent              261035 non-null  category      
 4   parent-name         261035 non-null  category      
 5   electricity_demand  261035 non-null  float64       
 6   value-units         261035 non-null  category      
 7   temperature_2m      261035 non-null  float64       
 8   hour                261035 non-null  int32         
 9   day                 261035 non-null  int3